<a href="https://colab.research.google.com/github/aminekhasteh/Breast_Cancer_Classification_Neural_Network/blob/master/Creating_main_data/creating_main_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
import os

In [194]:
city_names = {"ST. JOHN'S INTL A":"St.Johns","VANCOUVER INTL A":"Vancouver","EDMONTON NAMAO AWOS A":"Edmonton","QUEBEC INTL A":"Quebec City",
              "HAMILTON A":"Hamilton","WINDSOR A":"Windsor","VICTORIA HARBOUR A":"Victoria","TORONTO INTL A":"Toronto","WINNIPEG INTL A":"Winnipeg",
              "OTTAWA INTL A":"Ottawa","SASKATOON INTL A":"Saskatoon","KITCHENER/WATERLOO":"Kitchener/Waterloo","LONDON A":"London",
              "OTTAWA GATINEAU A":"Gatineau","CALGARY INTL A":"Calgary","MONTREAL INTL A":"Montreal","SHERBROOKE":"Sherbrooke","VICTORIA HARBOUR A":"Victoria",
              "OSHAWA":"Oshawa","REGINA INTL A":"Regina","HALIFAX STANFIELD INT'L A":"Halifax"}
province_names = {"Ottawa":"ON","Hamilton":"ON","Victoria":"BC","Saskatoon":"SK","Quebec City":"QC","St.Johns":"NL","Winnipeg":"MB",
                  "Kitchener/Waterloo":"ON","Calgary":"AB","London":"ON","Toronto":"ON","Windsor":"ON","Gatineau":"QC","Vancouver":"BC",
                  "Edmonton":"AB","Montreal":"QC","Sherbrooke":"QC","Oshawa":"ON","Regina":"SK"}
province_short = {"Newfoundland and Labrador":"NL","Prince Edward Island":"PE","Nova Scotia":"NS","New Brunswick":"NB",
                  "Quebec":"QC","Ontario":"ON","Manitoba":"MB","Saskatchewan":"SK","Alberta":"AB","British Columbia":"BC",
                  "Yukon Territory":"YT","Northwest Territories":"NT","Nunavut":"NU"}

# Merging the weather data for each city separately

In [118]:
path = '/content/drive/MyDrive/Colab Notebooks/Covid19/Weather_data/'

In [119]:
m = [s for s in os.listdir(path) if ".csv" in s]

In [120]:
mobility_dat = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Covid19/Mobility_data/2020_CA_Region_Mobility_Report.csv')

In [121]:
mobility_dat.head(700)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,CA,Canada,NaN,NaN,NaN,NaN,NaN,2020-02-15,4.0,2.0,10.0,3.0,1.0,0.0
1,CA,Canada,NaN,NaN,NaN,NaN,NaN,2020-02-16,13.0,8.0,41.0,4.0,0.0,-2.0
2,CA,Canada,NaN,NaN,NaN,NaN,NaN,2020-02-17,-12.0,-15.0,63.0,-28.0,-52.0,11.0
3,CA,Canada,NaN,NaN,NaN,NaN,NaN,2020-02-18,-1.0,4.0,6.0,-1.0,-1.0,1.0
4,CA,Canada,NaN,NaN,NaN,NaN,NaN,2020-02-19,1.0,1.0,9.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,CA,Canada,Alberta,Division No. 1 - Medicine Hat,NaN,NaN,NaN,2020-06-01,-16.0,29.0,NaN,-37.0,-32.0,7.0
696,CA,Canada,Alberta,Division No. 1 - Medicine Hat,NaN,NaN,NaN,2020-06-02,-17.0,29.0,NaN,-36.0,-30.0,8.0
697,CA,Canada,Alberta,Division No. 1 - Medicine Hat,NaN,NaN,NaN,2020-06-03,-16.0,32.0,NaN,-33.0,-29.0,9.0
698,CA,Canada,Alberta,Division No. 1 - Medicine Hat,NaN,NaN,NaN,2020-06-04,-16.0,31.0,NaN,-30.0,-31.0,8.0


In [122]:
mobility_dat['sub_region_1'].value_counts()

Quebec                       23098
Ontario                      14469
British Columbia              7451
Alberta                       5333
Saskatchewan                  4437
New Brunswick                 3940
Manitoba                      3759
Nova Scotia                   3558
Newfoundland and Labrador     1979
Prince Edward Island          1030
Nunavut                        436
Northwest Territories          269
Yukon                          249
Name: sub_region_1, dtype: int64

In [132]:
all_dat = pd.DataFrame()

In [131]:
merged_dat.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag,province,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,-78.88,43.92,Oshawa,6155875.0,2020-01-01,2020.0,1.0,1.0,NaN,0.3,NaN,-4.8,NaN,-2.3,NaN,20.3,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,27.0,NaN,41.0,NaN,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-78.88,43.92,Oshawa,6155875.0,2020-01-02,2020.0,1.0,2.0,NaN,5.4,NaN,-3.5,NaN,1.0,NaN,17.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,22.0,NaN,45.0,NaN,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-78.88,43.92,Oshawa,6155875.0,2020-01-03,2020.0,1.0,3.0,NaN,8.0,NaN,0.3,NaN,4.2,NaN,13.8,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,M,NaN,M,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-78.88,43.92,Oshawa,6155875.0,2020-01-04,2020.0,1.0,4.0,NaN,2.2,NaN,-0.8,NaN,0.7,NaN,17.3,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,M,NaN,M,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-78.88,43.92,Oshawa,6155875.0,2020-01-05,2020.0,1.0,5.0,NaN,1.0,NaN,-3.2,NaN,-1.1,NaN,19.1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,31.0,NaN,39.0,NaN,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
for item in m:
    tmp_dat = pd.read_csv(path+item)
    tmp_dat['Station Name'] = tmp_dat['Station Name'].replace(city_names)
    tmp_dat['province'] = tmp_dat['Station Name'].replace(province_names)
    merged_dat = pd.merge(left=tmp_dat, right=mobility_dat, how='outer', left_on=['Date/Time','province'], right_on=['date','sub_region_1'])
    all_dat = pd.concat([all_dat, merged_dat], axis=0)

In [134]:
print(all_dat.shape)

(1413360, 46)


In [146]:
all_dat['province'].value_counts()

ON         2562
QC         1464
BC          732
AB          732
SK          732
MB          366
NL          366
Halifax     366
Name: province, dtype: int64

In [148]:
all_dat = all_dat.dropna(subset=['province'])

In [149]:
print(all_dat.shape) # This number adds up now! (20 cities and 365 days for each)

(7320, 47)


In [150]:
all_dat = all_dat.reset_index()

In [151]:
# Changing the date column to date type
all_dat['Date/Time'] = all_dat['Date/Time'].astype('datetime64[ns]')

## Removing the variables we don't need

In [152]:
all_dat = all_dat.drop(['Longitude (x)', 'Latitude (y)','Climate ID','Data Quality','Max Temp Flag','Min Temp Flag','Mean Temp Flag',
                        'Heat Deg Days (°C)', 'Heat Deg Days Flag', 'Cool Deg Days (°C)', 'Cool Deg Days Flag', 'Total Rain Flag',
                        'Total Snow Flag', 'Total Precip Flag', 'Snow on Grnd Flag', 'Dir of Max Gust (10s deg)', 'Dir of Max Gust Flag',
                        'Spd of Max Gust (km/h)', 'Spd of Max Gust Flag', 'country_region_code', 'country_region', 'metro_area',
                        'iso_3166_2_code', 'census_fips_code','date'], axis=1)

In [153]:
print(all_dat.shape)

(7320, 23)


# Now, adding the intervention information from the COVID-19 Intervention Timeline in Canada website

In [158]:
# importing datetime module 
import datetime

In [159]:
path1 = '/content/drive/MyDrive/Colab Notebooks/Covid19/Intervention_data'

In [160]:
interven_dat = pd.read_csv(path1 +'/interventions.csv')

In [161]:
interven_dat['Intervention type'].value_counts()

Restrictions                              14
Supply management                          7
Other                                      5
Masks                                      4
Self-isolation                             3
Safety guidelines                          3
Change in practice                         2
Education                                  2
Recreation                                 1
Non-essential services                     1
Work from home                             1
Licence reinstatement/reclassification     1
Telemedicine/virtual care                  1
State of emergency                         1
Name: Intervention type, dtype: int64

In [163]:
detailed_description = interven_dat['Intervention type'] +'_'+ interven_dat['Description']
interven_dat['detailed_description']=detailed_description
interven_dat.detailed_description = interven_dat.detailed_description.str.replace(' ', '_')

In [164]:
interven_dat['Start date'] = interven_dat['Start date'].astype('datetime64[ns]')

In [165]:
interven_dat.head(50)

,Entry ID,Jurisdiction,Intervention category,Intervention type,Description,Organization,Start date,detailed_description
0,CAN004,Canada,Travel,Restrictions,New domestic transportation COVID-19 risk miti...,Transport Canada,2020-03-30,Restrictions_New_domestic_transportation_COVID...
1,CAN006,Canada,Public information,Masks,Recommendation for the use of face masks issued,Public Health Agency of Canada,2020-04-07,Masks_Recommendation_for_the_use_of_face_masks...
2,CAN014,Canada,Travel,Self-isolation,Requirement of a quarantine plan for all retur...,Prime Minister,2020-04-15,Self-isolation_Requirement_of_a_quarantine_pla...
3,CAN020,Canada,Public information,Masks,Mandatory use of face coverings for all air an...,Transport Canada,2020-04-17,Masks_Mandatory_use_of_face_coverings_for_all_...
4,CAN021,Canada,Travel,Restrictions,Travel advisory warning against non-essential ...,Government of Canada,2020-01-29,Restrictions_Travel_advisory_warning_against_n...
5,CAN022,Canada,Travel,Restrictions,Travel advisory warning against all travel to ...,Government of Canada,2020-02-29,Restrictions_Travel_advisory_warning_against_a...
6,CAN023,Canada,Travel,Restrictions,Travel advisory warning against all non-essent...,Government of Canada,2020-03-02,Restrictions_Travel_advisory_warning_against_a...
7,CAN024,Canada,Travel,Restrictions,Travel advisory warning against all travel to ...,Government of Canada,2020-03-06,Restrictions_Travel_advisory_warning_against_a...
8,CAN025,Canada,Travel,Restrictions,Travel advisory warning against all cruise shi...,Government of Canada,2020-03-09,Restrictions_Travel_advisory_warning_against_a...
9,CAN026,Canada,Distancing,Work from home,Recommendation for the adoption of work-from-h...,Government of Canada,2020-03-10,Work_from_home_Recommendation_for_the_adoption...


In [184]:
# Adding each intervention as a category to all_dat dataframe
for row in range(interven_dat.shape[0]):
    intervention = interven_dat.loc[row,'detailed_description'] # Getting the name of the intervention
    all_dat[intervention]= 0 # Adding the intervention name as a column to the main dataset
    time_intervention_happened = interven_dat.loc[row,'Start date'] # Getting the time for when the intervention happened
    for city in list(province_names.keys()):
        all_dat.loc[(all_dat['Date/Time'] > time_intervention_happened) & (all_dat['Station Name']==city), intervention] = 1 # Changing 0 to 1 for when the intervention happened

In [185]:
all_dat.shape

(7320, 69)

# Now we look into the Census data

In [191]:
# changing column name
all_dat = all_dat.rename(columns={'Station Name':'City'})
all_dat['City'].value_counts()

Saskatoon             366
Kitchener/Waterloo    366
Oshawa                366
Windsor               366
St.Johns              366
London                366
Ottawa                366
Winnipeg              366
Toronto               366
Edmonton              366
Quebec City           366
Victoria              366
Vancouver             366
Regina                366
Sherbrooke            366
Halifax               366
Gatineau              366
Montreal              366
Hamilton              366
Calgary               366
Name: City, dtype: int64

In [192]:
# Now let's add the pronice names:
all_dat['Province'] = all_dat['City'].replace(province_names)
all_dat['Province'].value_counts()

ON         2562
QC         1464
SK          732
AB          732
BC          732
Halifax     366
MB          366
NL          366
Name: Province, dtype: int64

In [195]:
# importing the Census data
col_names = ['GEO UID','Province name','Population and dwelling counts / Population, 2016',
             'Population and dwelling counts / Population density per square kilometre','Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes',
             'Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 15 to 64 years ; Both sexes',
             'Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 0 to 14 years ; Both sexes',
             'Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 65 years and over ; Both sexes',
             'Age & Sex - Both sexes / Average age of the population ; Both sexes',
             'Dwelling characteristics / Number of persons in private households',
             'Income - Total Sex / Total - Total income groups in 2015 for the population aged 15 years and over in private households - 100% data / Percentage with total income',
             'Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data',
             'Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data / Under $5,000',
             'Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data / $5,000 to $9,999',
             'Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Apartment in a building that has five or more storeys',
             'Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Single-detached house',
             'Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Other attached dwelling',
             'Knowledge of official language - Both sexes / Total - Knowledge of official languages for the total population excluding institutional residents - 100% data ; Both sexes / Neither English nor French ; Both sexes',
             'Knowledge of official language - Both sexes / Total - Knowledge of official languages for the total population excluding institutional residents - 100% data ; Both sexes',
             'Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data',
             'Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data / Total visible minority population',
             'Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data / Not a visible minority']
census_dat = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Covid19/Census_data/Census_data.csv', names=col_names, skiprows=1)

In [198]:
# chaning province name to their short form
census_dat['province'] = census_dat['Province name'].replace(province_short)

In [199]:
census_dat.head(20)

,GEO UID,Province name,"Population and dwelling counts / Population, 2016",Population and dwelling counts / Population density per square kilometre,Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes,Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 15 to 64 years ; Both sexes,Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 0 to 14 years ; Both sexes,Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 65 years and over ; Both sexes,Age & Sex - Both sexes / Average age of the population ; Both sexes,Dwelling characteristics / Number of persons in private households,Income - Total Sex / Total - Total income groups in 2015 for the population aged 15 years and over in private households - 100% data / Percentage with total income,Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data,"Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data / Under $5,000","Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data / $5,000 to $9,999",Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Apartment in a building that has five or more storeys,Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Single-detached house,Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Other attached dwelling,Knowledge of official language - Both sexes / Total - Knowledge of official languages for the total population excluding institutional residents - 100% data ; Both sexes / Neither English nor French ; Both sexes,Knowledge of official language - Both sexes / Total - Knowledge of official languages for the total population excluding institutional residents - 100% data ; Both sexes,Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data,Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data / Total visible minority population,Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data / Not a visible minority,province
0,1,CANADA,35151728.0,3.9,35151730.0,23376530.0,5839570.0,5935635.0,41.0,34460060.0,96.0,14072080.0,228785.0,200315.0,1391040.0,7541495.0,4949790.0,648975.0,34767250.0,34460065.0,7674585.0,26785480.0,CANADA
1,10,Newfoundland and Labrador,519716.0,1.4,519715.0,344245.0,74440.0,101030.0,43.7,512250.0,96.4,218670.0,1610.0,3325.0,755.0,160245.0,56250.0,820.0,515680.0,512250.0,11810.0,500440.0,NL
2,11,Prince Edward Island,142907.0,25.1,142905.0,92510.0,22685.0,27710.0,42.7,139690.0,97.0,59470.0,790.0,795.0,55.0,41160.0,15725.0,1250.0,141015.0,139685.0,6645.0,133045.0,PE
3,12,Nova Scotia,923598.0,17.4,923595.0,605950.0,133825.0,183820.0,43.5,908340.0,96.1,401990.0,6835.0,7755.0,21345.0,263470.0,102300.0,2735.0,912300.0,908340.0,58650.0,849690.0,NS
4,13,New Brunswick,747101.0,10.5,747100.0,487820.0,110500.0,148785.0,43.6,730705.0,95.9,319775.0,3935.0,7795.0,3950.0,221515.0,80515.0,2370.0,736285.0,730705.0,24540.0,706170.0,NB
5,24,Quebec,8164361.0,6.0,8164365.0,5335910.0,1333255.0,1495190.0,41.9,7965450.0,96.7,3531665.0,47165.0,56315.0,187800.0,1602680.0,1717680.0,75065.0,8066560.0,7965455.0,1032365.0,6933090.0,QC
6,35,Ontario,13448494.0,14.8,13448495.0,8988870.0,2207970.0,2251655.0,41.0,13242160.0,95.6,5169170.0,86720.0,70245.0,886705.0,2807380.0,1460200.0,326935.0,13312865.0,13242160.0,3885585.0,9356575.0,ON
7,46,Manitoba,1278365.0,2.3,1278365.0,835575.0,243825.0,198965.0,39.2,1240700.0,95.3,489050.0,9255.0,8175.0,3

In [200]:
# Now we add the provincial level info to our dataset from the Census website:
merged_all_dat = pd.merge(left=all_dat, right=census_dat, how='outer', left_on='Province', right_on='province')

In [202]:
# Dropping the additinional province and the language related columns:
merged_all_dat = merged_all_dat.drop(['Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data',	
                                      'Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data / Total visible minority population',
                                      'Aboriginals and Visible Minorities - Total Sex / Total - Visible minority for the population in private households - 25% sample data / Not a visible minority'],axis=1)

In [203]:
merged_all_dat.shape

(7327, 90)

In [204]:
merged_all_dat.iloc[350:450,:].head()
#Recreation_National_parks_and_other_national_sites_reopened_for_limited_services_and_operations

,level_0,index,City,Date/Time,Year,Month,Day,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),province_x,sub_region_1,sub_region_2,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Restrictions_New_domestic_transportation_COVID-19_risk_mitigation_measures_announced,Masks_Recommendation_for_the_use_of_face_masks_issued,Self-isolation_Requirement_of_a_quarantine_plan_for_all_returning_international_travellers_prior_to_entry_announced,Masks_Mandatory_use_of_face_coverings_for_all_air_and_rail_passengers_announced,Restrictions_Travel_advisory_warning_against_non-essential_travel_to_China_issued,Restrictions_Travel_advisory_warning_against_all_travel_to_the_cities_of_Daegu_and_Cheongdo_issued,Restrictions_Travel_advisory_warning_against_all_non-essential_travel_to_affected_regions_of_Italy_issued,Restrictions_Travel_advisory_warning_against_all_travel_to_Iran_issued,Restrictions_Travel_advisory_warning_against_all_cruise_ship_travel_issued,Work_from_home_Recommendation_for_the_adoption_of_work-from-home_policies_issued,Restrictions_Cruise_ship_season_postponed,Restrictions_Travel_advisory_warning_against_all_non-essential_travel_outside_Canada_issued,Restrictions_Travel_restrictions_on_the_entry_of_all_foreign_nationals_(except_from_the_United_States)_announced,State_of_emergency_State_of_emergency_declared,Masks_Guidance_on_use_of_non-medical_masks_published,Self-isolation_Mandatory_14-day_self-isolation_for_those_returning_from_international_travel_announced,Safety_guidelines_Guidance_to_optimize_the_use_of_masks_and_respirators_published,...,"Supply_management_Essential_health_service_personnel_deployed_to_remote_First_Nations_communities_in_Ontario,_Manitoba_and_Alberta",Other_Use_of_first_COVID-19_serological_test_authorized,Recreation_National_parks_and_other_national_sites_reopened_for_limited_services_and_operations,Restrictions_Travel_restrictions_on_the_entry_of_foreign_nationals_from_the_United_States_announced,"Restrictions_Restriction_on_passenger_vessels_(except_cruise_ships)_operating_in_inland_rivers/lakes_in_N.W.T.,_Nun._and_Y.T._lifted",Masks_Mandatory_use_of_face_coverings_for_transportation_sector_workers_announced,Restrictions_Travel_restrictions_on_foreign_nationals_with_immediate_family_in_Canada_relaxed,Other_Launch_of_national_contact_tracing_app_announced,Other_Phase_1_of_mandatory_air_traveller_screening_announced,Licence_reinstatement/reclassification_Maintenance_of_certification_requirements_waived_for_physician_assistants,Non-essential_services_Opening_of_Service_Canada_Centres_announced,Restrictions_Restrictions_and_entry_conditions_for_travellers_transiting_through_Canada_to_Alaska_announced,"Other_Canada’s_Flight_Plan_for_Navigating_COVID-19,_aimed_at_reducing_risks_of_COVID-19_while_travelling_by_aircraft,_announced",Education_$112_million_in_funding_announced_to_support_a_safe_return_to_elementary_and_secondary_school_on_reserves,Education_$2_billion_in_funding_announced_for_provinces_and_territories_through_the_Safe_Return_To_Class_Fund,Other_Consistent_approach_for_air_carriers_to_collect_and_share_passenger_information_for_COVID-19_tracing_announced,"Supply_management_Deployment_of_Canadian_Rangers_to_communities_in_Saskatchewan,_northern_Ontario,_B.C.,_Yukon,_Northwest_Territories_announced",Safety_guidelines_Creation_of_Essential_Services_Contingency_Reserve_that_complements_existing_personal_protective_equipment_support_for_front-line_health_care_workers_announced,Safety_guidelines_Guidance_for_health_workers_in_acute_health_care_settings_published,Province,GEO UID,Province name,"Population and dwelling counts / Population, 2016",Population and dwelling counts / Population density per square kilometre,Age & Sex - Both sex

# Saving the final data

In [ ]:
# Full dataset:
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Covid19/Final_data/' + 'main_data_frame_mobility_intervention_weather_census.csv'
merged_all_dat.to_csv(path_to_save, header=True, index=False)

In [ ]:
# Only the interventions:
interven_dat_full = merged_all_dat[['City','Date/Time','Year','Month','Day','Restrictions_New_domestic_transportation_COVID-19_risk_mitigation_measures_announced',	
                                    'Masks_Recommendation_for_the_use_of_face_masks_issued','Self-isolation_Requirement_of_a_quarantine_plan_for_all_returning_international_travellers_prior_to_entry_announced',
                                    'Masks_Mandatory_use_of_face_coverings_for_all_air_and_rail_passengers_announced','Restrictions_Travel_advisory_warning_against_non-essential_travel_to_China_issued',
                                    'Restrictions_Travel_advisory_warning_against_all_travel_to_the_cities_of_Daegu_and_Cheongdo_issued	Restrictions_Travel_advisory_warning_against_all_non-essential_travel_to_affected_regions_of_Italy_issued',
                                    'Restrictions_Travel_advisory_warning_against_all_travel_to_Iran_issued',
                                    'Restrictions_Travel_advisory_warning_against_all_cruise_ship_travel_issued	Work_from_home_Recommendation_for_the_adoption_of_work-from-home_policies_issued',
                                    'Restrictions_Cruise_ship_season_postponed',
                                    'Restrictions_Travel_advisory_warning_against_all_non-essential_travel_outside_Canada_issued',
                                    'Restrictions_Travel_restrictions_on_the_entry_of_all_foreign_nationals_(except_from_the_United_States)_announced',
                                    'State_of_emergency_State_of_emergency_declared	Masks_Guidance_on_use_of_non-medical_masks_published',
                                    'Self-isolation_Mandatory_14-day_self-isolation_for_those_returning_from_international_travel_announced',
                                    'Safety_guidelines_Guidance_to_optimize_the_use_of_masks_and_respirators_published',
                                    'Supply_management_Deployment_of_Canadian_Rangers_to_Inuit_communities_in_northern_Quebec_announced',
                                    'Restrictions_Restrictions_on_commercial_marine_activity_announced',
                                    'Restrictions_Travel_restrictions_on_the_entry_of_foreign_nationals_from_the_United_States_announced',
                                    'Restrictions_Restriction_on_passenger_vessels_(except_cruise_ships)_operating_in_inland_rivers/lakes_in_N.W.T.,_Nun._and_Y.T._lifted',
                                    'Masks_Mandatory_use_of_face_coverings_for_transportation_sector_workers_announced',
                                    'Restrictions_Travel_restrictions_on_foreign_nationals_with_immediate_family_in_Canada_relaxed',
                                    'Other_Launch_of_national_contact_tracing_app_announced	Other_Phase_1_of_mandatory_air_traveller_screening_announced',
                                    'Licence_reinstatement/reclassification_Maintenance_of_certification_requirements_waived_for_physician_assistants',
                                    'Non-essential_services_Opening_of_Service_Canada_Centres_announced',
                                    'Restrictions_Restrictions_and_entry_conditions_for_travellers_transiting_through_Canada_to_Alaska_announced',
                                    'Other_Canada’s_Flight_Plan_for_Navigating_COVID-19,_aimed_at_reducing_risks_of_COVID-19_while_travelling_by_aircraft,_announced',
                                    'Education_$112_million_in_funding_announced_to_support_a_safe_return_to_elementary_and_secondary_school_on_reserves',	
                                    'Education_$2_billion_in_funding_announced_for_provinces_and_territories_through_the_Safe_Return_To_Class_Fund',
                                    'Other_Consistent_approach_for_air_carriers_to_collect_and_share_passenger_information_for_COVID-19_tracing_announced',
                                    'Supply_management_Deployment_of_Canadian_Rangers_to_communities_in_Saskatchewan,_northern_Ontario,_B.C.,_Yukon,_Northwest_Territories_announced',
                                    'Safety_guidelines_Creation_of_Essential_Services_Contingency_Reserve_that_complements_existing_personal_protective_equipment_support_for_front-line_health_care_workers_announced',
                                    'Safety_guidelines_Guidance_for_health_workers_in_acute_health_care_settings_published',
                                    'Province']]
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Covid19/Final_data/' + 'main_data_frame_intervention.csv'
interven_dat_full.to_csv(path_to_save, header=True, index=False)                              

In [ ]:
# Only Census data:
census_dat_full = merged_all_dat[['City','Date/Time','Year','Month','Day','Province','Population and dwelling counts / Population, 2016',
             'Population and dwelling counts / Population density per square kilometre','Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes',
             'Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 15 to 64 years ; Both sexes',
             'Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 0 to 14 years ; Both sexes',
             'Age & Sex - Both sexes / Total - Age groups and average age of the population - 100% data ; Both sexes / 65 years and over ; Both sexes',
             'Age & Sex - Both sexes / Average age of the population ; Both sexes',
             'Dwelling characteristics / Number of persons in private households',
             'Income - Total Sex / Total - Total income groups in 2015 for the population aged 15 years and over in private households - 100% data / Percentage with total income',
             'Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data',
             'Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data / Under $5,000',
             'Income - Total Sex / Total - Household total income groups in 2015 for private households - 100% data / $5,000 to $9,999',
             'Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Apartment in a building that has five or more storeys',
             'Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Single-detached house',
             'Dwelling characteristics / Total - Occupied private dwellings by structural type of dwelling - 100% data / Other attached dwelling',
             'Knowledge of official language - Both sexes / Total - Knowledge of official languages for the total population excluding institutional residents - 100% data ; Both sexes / Neither English nor French ; Both sexes',
             'Knowledge of official language - Both sexes / Total - Knowledge of official languages for the total population excluding institutional residents - 100% data ; Both sexes']]
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Covid19/Final_data/' + 'main_data_frame_census.csv'
census_dat_full.to_csv(path_to_save, header=True, index=False)                              

In [ ]:
# Only weather data:
weather_dat_full = merged_all_dat[['City','Date/Time','Year','Month','Day','Province', 'Max Temp (°C)',
                                   'Min Temp (°C)',	'Mean Temp (°C)','Total Rain (mm)','Total Snow (cm)',
                                   'Total Precip (mm)','Snow on Grnd (cm)']]
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Covid19/Final_data/' + 'main_data_frame_weather.csv'
weather_dat_full.to_csv(path_to_save, header=True, index=False)  

In [ ]:
# Only mobility data:
mobility_dat_full = merged_all_dat[['City','Date/Time','Year','Month','Day','Province', 
                                    'retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline',
                                    'parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline',
                                    'workplaces_percent_change_from_baseline','residential_percent_change_from_baseline']]
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Covid19/Final_data/' + 'main_data_frame_mobility.csv'
mobility_dat_full.to_csv(path_to_save, header=True, index=False)  